In [1]:
import pandas as pd

from lib.utils import get_id_nums
from lib.clean_consignee import clean_consignee
from lib.clean_shipper import clean_shipper

## Set up countries table

- Get table of countries with alpha-2 code that includes region from repository
    - https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes
- Remove the unnecessary columns, change the index field and rename to id
- Set up lookup table to convert country names to country codes

In [2]:
countries_cols = ['name', 'alpha-2', 'region', 'sub-region']
# keep_default_na=False prevents pandas from interpreting Namibia's alpha-2 (NA) as NaN
countries = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/all.csv', keep_default_na=False, usecols=countries_cols)

# Change countries index column to be alpha-2 values and rename to id
countries.set_index('alpha-2', inplace=True)
countries.index.name = 'id'

# Set for O(1) look up during data cleaning
alpha_2_set = set(countries.index)

# Add dictionary for O(1) look up during cleaning
country_dict = {x[1].upper(): x[0] for x in countries.itertuples()}
# Add a few of the statiscally significant enough outliers - 
    # cheaper computationally than performing trims on each row (I think)
country_dict['TAIWAN'] = 'TW'
country_dict['SOUTH KOREA'] = 'KR'
country_dict['SHANGHAI CN'] = 'CN'
country_dict['SHANGHAI'] = 'CN'
country_dict['SHANGHAI .'] = 'CN'
country_dict['HONG KONG .'] = 'CN'
country_dict['TAIPEI .'] = 'TW'

- Convert altered countries dataframe to csv

In [8]:
countries_file_path = '/Users/jesseputnam/cs-learning/skillstorm/project01/data/final/countries.csv'
countries.to_csv(countries_file_path)


## Consolidate Shipper CSV Files

In [3]:
# Choose columns to keep
shipper_keep_cols = ['identifier', 'shipper_party_name', 'shipper_party_address_1', 'shipper_party_address_2', 'shipper_party_address_3', 'shipper_party_address_4', 'country_code']

shipper_raw_path = '/Users/jesseputnam/cs-learning/skillstorm/project01/data/final_raw/shipper.csv'

- Read CSVs to DataFrames with only necessary columns

In [ ]:
shipper_0 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2018/shipper_2018_part_0.csv', usecols=shipper_keep_cols)
shipper_1 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2018/shipper_2018_part_1.csv', usecols=shipper_keep_cols)
shipper_2 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2019/shipper_2019_part_0.csv', usecols=shipper_keep_cols)
shipper_3 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2019/shipper_2019_part_1.csv', usecols=shipper_keep_cols)
shipper_4 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2020/shipper_2020_part_0.csv', usecols=shipper_keep_cols)
shipper_5 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2020/shipper_2020_part_1.csv', usecols=shipper_keep_cols)

- Concatenate shippers DataFrames to single DataFrame

In [ ]:
shippers = pd.concat([shipper_0, shipper_1, shipper_2, shipper_3, shipper_4, shipper_5], ignore_index=True)

## Set up Shipper table

In [4]:

#TODO change to unified shipper dataframe
shipper_2018_0 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2018/shipper_2018_part_0.csv', index_col=False, usecols=shipper_keep_cols)
shipper_2018_1 = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2018/shipper_2018_part_1.csv', usecols=shipper_keep_cols)

# Replace NaN in name with Unknown
shipper_2018_0['shipper_party_name'].fillna('N/A', inplace=True)

- Clean and consolidate 

In [5]:

#TODO Clean and consolidate on shipper
shipper_2018_0_clean = shipper_2018_0.apply(lambda row: clean_shipper(row, alpha_2_set, country_dict), axis=1)
shipper_2018_0_clean = shipper_2018_0_clean[['identifier', 'shipper_party_name', 'country_code']]

- Map names to ids

In [7]:

#TODO Map IDs on common shipper names
shipper_id_dict = get_id_nums(shipper_2018_0_clean['shipper_party_name'])
shipper_2018_0_clean['id'] = shipper_2018_0_clean['shipper_party_name'].map(shipper_id_dict)

## Consignee

In [11]:
consignee_cols = ['identifier', 'consignee_name', 'consignee_address_1', 'country_code']

#TODO
#TODO Upload consignees
consignee_full = pd.read_csv('/Users/jesseputnam/cs-learning/skillstorm/project01/data/2018/consignee_2018_part_0.csv', usecols=consignee_cols)

#TODO Concatenate to single DataFrame

#TODO Fill NaN names with N/A
consignee_full['consignee_name'].fillna('N/A', inplace=True)

#TODO Clean names and country codes
consignee_full_clean = consignee_full.apply(lambda row: clean_consignee(row, alpha_2_set), axis=1)

#TODO Map IDs by common consignee names
consignee_id_dict = get_id_nums(consignee_full_clean['consignee_name'])
consignee_full_clean['id'] = consignee_full_clean['consignee_name'].map(consignee_id_dict)